# Compiling NIFTI files from tiff stack files

For the compatibility in the following ANTs registration,  
the images taken from the other side to the main angle are flipped and re-oriented.  

You can interactively set the annotation phrase.  
For example, if there are files like the following:  
- descSPIM_angle1_GFP_target_signal_some_notes.tif
- descSPIM_angle2_GFP_target_signal_some_notes.tif
- descSPIM_angle1_PI_nuclear_stain_some_notes.tif
- descSPIM_angle2_PI_nuclear_stain_some_notes.tif  

**To set the 'angle1' as the main angle, enter in the message box 'angle2'**.  
All the images that contains 'angle2' will be flipped both horizontally and along z axis, so that the compiled NIFTI files can be used in the following ANTs registration steps.

In [ ]:
"""
TIFF Stack to NIfTI Converter
------------------------------
This script provides utility functions to convert TIFF stacks to NIfTI format (.nii.gz).
It also includes functionalities to flip the TIFF stacks based on a given keyword, normalize and scale image values, and save both full-resolution and downsized NIfTI images.

Dependencies:
- os
- glob
- tifffile
- numpy
- nibabel
- skimage.transform
- multiprocessing
- concurrent.futures

Author: takakiom
Date: 09/19/2023
License: MIT License (refer to the LICENSE file in the root directory for more details)
"""

import os
import glob
import tifffile
import numpy as np
import nibabel as nb
from skimage.transform import rescale
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

def load_tiff_stack(imgpath):
    stack = tifffile.imread(imgpath)
    return stack

def flip_tiff_stack(stack):
    return np.flip(np.flip(stack, axis=2), axis=0)

def normalize_and_scale(img):
    img = (img - img.min()) / (img.max() - img.min())
    img = (img * 255).astype(np.uint8)
    return img

def save_nifti(stack, niftiname, spx, spy, spz):
    stack = np.swapaxes(stack,0,2)
    nim = nb.Nifti1Image(stack, affine=None)
    aff = np.diag([-spx,-spy,spz,1])
    nim.header.set_qform(aff, code=2)
    nim.to_filename(niftiname)

def process_folder(base_dir, filename, flip_key, spx, spy, spz):
    file_name = os.path.splitext(os.path.basename(filename))[0]

    full_stack = load_tiff_stack(filename)
    if flip_key in file_name:
        file_name = file_name.replace(flip_key, f'Flipped{flip_key}')
        full_stack = flip_tiff_stack(full_stack)
    # Save full resolution nifti
    save_nifti(full_stack, os.path.join(base_dir, f'{file_name}_fullsize.nii.gz'), spx, spy, spz)
    # Save downsize nifti
    down_stack = full_stack[::4, :, :]
    down_stack = rescale(down_stack, (1, 0.5, 0.5), anti_aliasing=True, channel_axis=None)
    down_stack = normalize_and_scale(down_stack)

    save_nifti(down_stack, os.path.join(base_dir, f'{file_name}_downsize.nii.gz'), spx*2, spy*2, spz*4)

if __name__ == "__main__":
    spx=float(3.45)
    spy=float(3.45)
    spz=float(10)

    base_dir = input("Please enter base folder path: ")
    flip_key = input("Enter the annotation for the angle180 (e.g.;angle180, Angle180, angle2 etc.) :")

    file_list = glob.glob(os.path.join(base_dir, "*.tif"))

    with ProcessPoolExecutor(max_workers=mp.cpu_count()) as executor:
        executor.map(process_folder, [base_dir]*len(file_list), file_list, [flip_key]*len(file_list), [spx]*len(file_list), [spy]*len(file_list), [spz]*len(file_list))
